In [1]:
import urllib2
import os
import numpy as np
import pandas as pd
from IPython.core.pylabtools import figsize
from bs4 import BeautifulSoup

## Utility Functions

These will be used to wrangle the marathon data into a form ripe for analysis.

In [2]:
def convert_time_to_h(time):
    """Convert marathon format time (hh:mm:ss) to hours"""
    
    try:
        time = map(int, time.split(':'))
        time_s = time[0] + time[1]/60. + time[2]/3600.
    except (ValueError, AttributeError):
        time_s = None
    
    return(time_s)

In [3]:
def process_lon_runner_field(runner):
    """
    Outputs a runner name and representative country from the 
    London Marathon runner field.
    """
    fields = re.findall("[a-z]+", runner.lower())
    country = fields[-1]
    name = ' '.join(fields[:-1])
    
    return(pd.Series({"name": name, "country": country}))

In [4]:
def split_age_field(agegender):
    """
    Splits concatenated age field in raw NYC marathon data into independent 
    age, gender, and age category fields. 
    """
    
    age, gender = int(agegender[:-1]), agegender[-1].lower()
    return (pd.Series({'age': age, 'gender': gender}))

## London Marathon Data

The 2015 London Marathon results could be easily scraped. I just built a simple scraper.

In [ ]:
def get_london_marathon_data():
    """Scrape raw London Marathon 2015 results from official site"""
    
    url = "http://results-2015.virginmoneylondonmarathon.com/2015/?page={0}&event=MAS&num_results=1000&pid=search&search%5Bsex%5D=%25&search%5Bnation%5D=%25&search_sort=name"

    for page_num in range(1, 45): 
        runner_data = urllib2.urlopen(url.format(page_num))    
        file_str = u"london_marathon_result_{0}.txt".format(page_num)
        with open(file_str, 'wb') as txt_file:
            txt_file.write(runner_data.read())

    return 

# get_london_marathon_data()

In [ ]:
def tabularize_lon_data():
    """Turn all london marathon files into one dataframe after processing html"""
    
    file_str = "london_marathon_result_{0}.txt"
    rows = []

    for page_num in range(1, 45): 
        with open(file_str.format(page_num), "rb") as txt_file:
            run_data = txt_file.read()

        soup = BeautifulSoup(run_data, "html5lib")
        table = soup.find("table", attrs={"class": "list-table"})

        for tr in table.find("tbody").find_all("tr"):
            row = [td.get_text() for td in tr.find_all("td")]
            rows.append(row)

    lon_data = pd.DataFrame(rows, columns=['0', '1', '2', 'runner', 'club', 'runner_num', 'age_cat', 'half', 'finish', '3'])
    
    return (lon_data)

In [ ]:
lon_data = tabularize_lon_data()

assert lon_data.shape[0] > 40000, "Missing a few rows?"

# Clean up dataframe 
lon_data.drop(['0', '1', '2', '3', 'club', 'runner_num'], axis=1, inplace=True)
lon_data['13.1_mi'] = lon_data.half.apply(convert_time_to_h)
lon_data['26.2_mi'] = lon_data.finish.apply(convert_time_to_h)
lon_data = lon_data.join(lon_data.runner.apply(process_lon_runner_field))
lon_data = lon_data[['name', 'country', 'age_cat', '13.1_mi', '26.2_mi']]

In [ ]:
# Create same age categories in NYC Marathon data for comparison
print (lon_data.age_cat.unique())
lon_data.tail(5)

Pickle London marathon data for further analysis

In [ ]:
lon_data.to_pickle("london_marathon_2015_processed.pickle")

## NYC Marathon Data

In [5]:
def tabularize_nyc_data():
    """
    Read all NYC Marathon data files and output a pandas dataframe 
    """

    nyc_data_filenames = [f for f in os.listdir(".") if "nyc_marathon_ages" in f]

    # Relevant columns to the analysis but they're name named properly
    cols_rel = ['First Name', 'Last Name', 'Age', 'State/Country', 'FinishTime', 
                '5 km', '10 km', '15 km', '20 km', '13.1 mi', '25 km', '30 km', '35 km', '40 km']
    cols_names = ['first_name', 'last_name', 'agegender', 'state', 'country', 'finish_time',
                  '5 km', '10 km', '15 km', '20 km', '13.1 mi', '25 km', '30 km', '35 km']

    nyc_data = pd.DataFrame()
    for nyc_data_filename in nyc_data_filenames:
        data = pd.read_csv(nyc_data_filename)
        data = data[cols_rel]
        data.columns = cols_names
        data.dropna(subset=['first_name'], inplace=True)
        nyc_data = pd.concat([nyc_data, data], ignore_index=True)

    assert nyc_data.shape[0] > 40000, "Missing some rows?"
    
    return (nyc_data)

In [6]:
nyc_data = tabularize_nyc_data()
nyc_data['name'] = nyc_data.last_name.str.lower() + ' ' + nyc_data.first_name.str.lower()
nyc_data['country'] = nyc_data.country.str.lower()
nyc_data['state'] = nyc_data.state.str.lower()
nyc_data['26.2_mi'] = nyc_data.finish_time.apply(convert_time_to_h)
nyc_data['5_km'] = nyc_data["5 km"].apply(convert_time_to_h)
nyc_data['10_km'] = nyc_data["10 km"].apply(convert_time_to_h)
nyc_data['15_km'] = nyc_data["15 km"].apply(convert_time_to_h)
nyc_data['20_km'] = nyc_data["20 km"].apply(convert_time_to_h)
nyc_data['13.1_mi'] = nyc_data["13.1 mi"].apply(convert_time_to_h)
nyc_data['25_km'] = nyc_data["25 km"].apply(convert_time_to_h)
nyc_data['30_km'] = nyc_data["30 km"].apply(convert_time_to_h)
nyc_data['35_km'] = nyc_data["35 km"].apply(convert_time_to_h)
nyc_data = nyc_data.join(nyc_data.agegender.apply(split_age_field))
nyc_data = nyc_data[['name', 'state', 'country', 'age', 'gender', 
                     '26.2_mi', '5_km', '10_km', '15_km', '20_km', '13.1_mi',
                     '25_km', '30_km', '35_km']]

Pickle NYC 2015 Marathon data for further analysis

In [8]:
nyc_data.to_pickle("nyc_marathon_2015_processed.pickle")